In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/yelp-dataset/Dataset_User_Agreement.pdf
/kaggle/input/yelp-dataset/yelp_academic_dataset_review.json
/kaggle/input/yelp-dataset/yelp_academic_dataset_checkin.json
/kaggle/input/yelp-dataset/yelp_academic_dataset_business.json
/kaggle/input/yelp-dataset/yelp_academic_dataset_tip.json
/kaggle/input/yelp-dataset/yelp_academic_dataset_user.json


In [2]:
from sqlalchemy import create_engine
!pip install PyMySql
import pymysql
import sqlalchemy
import json

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 2.6 MB/s eta 0:00:00


In [4]:
username = 'A04' # Your team name
password = 'A04password'
host = 'testproject.mysql.database.azure.com'
database = 'A04'  # Your team name

connection_string = f"mysql+pymysql://{username}:{password}@{host}/{database}"

ssl_args = {
    'ssl_ca': 'path_to/ca-cert.pem',
    'ssl_cert': 'path_to/client-cert.pem',
    'ssl_key': 'path_to/client-key.pem'
}

engine = create_engine(
    connection_string,
    connect_args={'ssl': ssl_args}
)

In [ ]:
tip = []
with open('/kaggle/input/yelp-dataset/yelp_academic_dataset_tip.json') as fl:
    for i, line in enumerate(fl):
        tip.append(json.loads(line))
dftip = pd.DataFrame(tip)
dftip.head()

dftip.to_sql(name = "tips", con = engine, if_exists='replace', index=True, chunksize=50000)

In [ ]:
user = []
with open('/kaggle/input/yelp-dataset/yelp_academic_dataset_user.json') as fl:
    for i, line in enumerate(fl):
        user.append(json.loads(line))
dfuser = pd.DataFrame(user)
dfuser.head()

dfuser.to_sql(name = "users", con = engine, if_exists='replace', index=True, dtype={'friends': sqlalchemy.types.JSON}, chunksize=50000)

In [ ]:
business = []
with open('/kaggle/input/yelp-dataset/yelp_academic_dataset_business.json') as fl:
    for i, line in enumerate(fl):
        business.append(json.loads(line))
dfbusiness = pd.DataFrame(business)
dfbusiness.head()

dfbusiness.to_sql(name = "business", con = engine, if_exists='replace', index=True, dtype={'attributes': sqlalchemy.types.JSON, 'hours': sqlalchemy.types.JSON}, chunksize=50000)

In [ ]:
checkin = []
with open('/kaggle/input/yelp-dataset/yelp_academic_dataset_checkin.json') as fl:
    for i, line in enumerate(fl):
        checkin.append(json.loads(line))
dfcheckin = pd.DataFrame(checkin)
dfcheckin.head()

dfcheckin.to_sql(name = "checkin", con = engine, if_exists='replace', index=True, dtype={'date': sqlalchemy.types.JSON}, chunksize=50000)

In [ ]:
review = []
with open('/kaggle/input/yelp-dataset/yelp_academic_dataset_review.json') as fl:
    for i, line in enumerate(fl):
        review.append(json.loads(line))
dfreview = pd.DataFrame(review)
dfreview.head()

dfreview.to_sql(name = "review", con = engine, if_exists='replace', index=True, chunksize=50000)

In [ ]:
def expand_multivalue_column(df, col_name):
    # Convert string representation of dictionaries into actual dictionaries (if needed)
    expanded_df = df[col_name].apply(pd.Series)
    
    # For columns within the expanded dataframe that are still string representations of dictionaries
    for sub_col in expanded_df.columns:
        if isinstance(expanded_df[sub_col].iloc[0], str) and expanded_df[sub_col].iloc[0].startswith("{"):
            sub_df = expanded_df[sub_col].map(eval).apply(pd.Series)
            sub_df = sub_df.add_prefix(f'{sub_col}_')
            expanded_df = pd.concat([expanded_df.drop([sub_col], axis=1), sub_df], axis=1)
    
    # Add the business_id column for the foreign key reference
    expanded_df['business_id'] = df['business_id']
    
    return expanded_df

# Expand the attributes and hours columns into separate tables
attributes_table = expand_multivalue_column(business_df, 'attributes')
hours_table = expand_multivalue_column(business_df, 'hours')

# Drop the 'attributes' and 'hours' columns from the original business_df
business_df = business_df.drop(columns=['attributes', 'hours'])

print("Original Business Table:")
print(business_df)
print("\nAttributes Table:")
print(attributes_table)
print("\nHours Table:")
print(hours_table)

In [ ]:
# Split the 'categories' column by commas and transform to long format
categories_table = business_df['categories'].str.split(', ', expand=True).stack().reset_index()
categories_table.columns = ['business_id', 'category_number', 'category']
categories_table = categories_table[['business_id', 'category']]  # Drop the category_number column

# Drop the 'categories' column from the original business_df
business_df = business_df.drop(columns=['categories'])

print("Original Business Table:")
print(business_df)
print("\nCategories Table:")
print(categories_table)

In [ ]:
business_df.to_sql(name = "business", con = engine, if_exists='replace', index=True, chunksize=50000)

In [ ]:
attributes_table.to_sql(name = "attributes", con = engine, if_exists='replace', index=True, chunksize=50000)

In [ ]:
hours_table.to_sql(name = "hours", con = engine, if_exists='replace', index=True, chunksize=50000)

In [ ]:
categories_table.to_sql(name = "categories", con = engine, if_exists='replace', index=True, chunksize=50000)

In [6]:
business = []
with open('/kaggle/input/yelp-dataset/yelp_academic_dataset_business.json') as fl:
    for i, line in enumerate(fl):
        business.append(json.loads(line))
dfbusiness = pd.DataFrame(business)
        

In [7]:
print(dfbusiness.head())

              business_id                      name  \
0  Pns2l4eNsfO8kk83dixA6A  Abby Rappoport, LAC, CMQ   
1  mpf3x-BjTdTEA3yCZrAYPw             The UPS Store   
2  tUFrWirKiKi_TAnsVWINQQ                    Target   
3  MTSW4McQd7CbVtyjqoe9mw        St Honore Pastries   
4  mWMc6_wTdE0EUBKIGXDVfA  Perkiomen Valley Brewery   

                           address           city state postal_code  \
0           1616 Chapala St, Ste 2  Santa Barbara    CA       93101   
1  87 Grasso Plaza Shopping Center         Affton    MO       63123   
2             5255 E Broadway Blvd         Tucson    AZ       85711   
3                      935 Race St   Philadelphia    PA       19107   
4                    101 Walnut St     Green Lane    PA       18054   

    latitude   longitude  stars  review_count  is_open  \
0  34.426679 -119.711197    5.0             7        0   
1  38.551126  -90.335695    3.0            15        1   
2  32.223236 -110.880452    3.5            22        0   
3  39.9555

In [9]:
def new_exp_column(df, col_name):
    expand_df = df[col_name].apply(pd.Series)
    for i in expand_df.columns:
        if isinstance(expand_df[i].iloc[0], str) and expand_df[i].iloc[0].startswith("{"):
            new_df = expand_df[i].map(eval).apply(pd.Series)
            new_df = new_df.add_prefix(f'{i}_')  # Corrected this line
            expand_df = pd.concat([expand_df.drop([i], axis=1), new_df], axis=1)
    expand_df['business_id'] = df['business_id']  # Corrected column name here
 
    return expand_df
 
# Assuming 'business_df' is your original DataFrame
# Expand the attributes and hours columns into separate tables
attributes_df = new_exp_column(dfbusiness, 'attributes')
 
# Drop the 'attributes' and 'hours' columns from the original business_df
dfbusiness = dfbusiness.drop(columns=['attributes', 'hours'])
 
print("Original Business Table:")
print(dfbusiness)
print("\nAttributes Table:")
print(attributes_df)

Original Business Table:
                   business_id                      name  \
0       Pns2l4eNsfO8kk83dixA6A  Abby Rappoport, LAC, CMQ   
1       mpf3x-BjTdTEA3yCZrAYPw             The UPS Store   
2       tUFrWirKiKi_TAnsVWINQQ                    Target   
3       MTSW4McQd7CbVtyjqoe9mw        St Honore Pastries   
4       mWMc6_wTdE0EUBKIGXDVfA  Perkiomen Valley Brewery   
...                        ...                       ...   
150341  IUQopTMmYQG-qRtBk-8QnA              Binh's Nails   
150342  c8GjPIOTGVmIemT7j5_SyQ      Wild Birds Unlimited   
150343  _QAMST-NrQobXduilWEqSw         Claire's Boutique   
150344  mtGm22y5c2UHNXDFAjaPNw  Cyclery & Fitness Center   
150345  jV_XOycEzSlTx-65W906pg                   Sic Ink   

                                address           city state postal_code  \
0                1616 Chapala St, Ste 2  Santa Barbara    CA       93101   
1       87 Grasso Plaza Shopping Center         Affton    MO       63123   
2                  5255 E 

In [11]:
import ast
Parking_df = pd.DataFrame(attributes_df['BusinessParking'])
def safe_eval(text):
    try:
        return ast.literal_eval(text)
    except (SyntaxError, ValueError):
        return None
 
# Apply the custom function to handle the text data
Parking_df['BusinessParking'] = Parking_df['BusinessParking'].apply(safe_eval)
Parking_df = pd.concat([Parking_df, Parking_df['BusinessParking'].apply(pd.Series)], axis=1)
Parking_df = Parking_df.drop(columns='BusinessParking')
Parking_df['business_id'] = attributes_df['business_id']
print(Parking_df)

       garage street validated    lot  valet             business_id
0         NaN    NaN       NaN    NaN    NaN  Pns2l4eNsfO8kk83dixA6A
1         NaN    NaN       NaN    NaN    NaN  mpf3x-BjTdTEA3yCZrAYPw
2       False  False     False   True  False  tUFrWirKiKi_TAnsVWINQQ
3       False   True     False  False  False  MTSW4McQd7CbVtyjqoe9mw
4        None   None      None   True  False  mWMc6_wTdE0EUBKIGXDVfA
...       ...    ...       ...    ...    ...                     ...
150341    NaN    NaN       NaN    NaN    NaN  IUQopTMmYQG-qRtBk-8QnA
150342  False  False     False   True  False  c8GjPIOTGVmIemT7j5_SyQ
150343    NaN    NaN       NaN    NaN    NaN  _QAMST-NrQobXduilWEqSw
150344  False  False     False   True  False  mtGm22y5c2UHNXDFAjaPNw
150345  False  False     False   True  False  jV_XOycEzSlTx-65W906pg

[150346 rows x 6 columns]


In [12]:
Ambience_df = pd.DataFrame(attributes_df['Ambience'])
 
# Apply the custom function to handle the text data
Ambience_df['Ambience'] = Ambience_df['Ambience'].apply(safe_eval)
Ambience_df = pd.concat([Ambience_df, Ambience_df['Ambience'].apply(pd.Series)], axis=1)
Ambience_df = Ambience_df.drop(columns='Ambience')
Ambience_df['business_id'] = attributes_df['business_id']
print(Ambience_df)

       romantic intimate touristy hipster divey classy trendy upscale casual  \
0           NaN      NaN      NaN     NaN   NaN    NaN    NaN     NaN    NaN   
1           NaN      NaN      NaN     NaN   NaN    NaN    NaN     NaN    NaN   
2           NaN      NaN      NaN     NaN   NaN    NaN    NaN     NaN    NaN   
3           NaN      NaN      NaN     NaN   NaN    NaN    NaN     NaN    NaN   
4           NaN      NaN      NaN     NaN   NaN    NaN    NaN     NaN    NaN   
...         ...      ...      ...     ...   ...    ...    ...     ...    ...   
150341      NaN      NaN      NaN     NaN   NaN    NaN    NaN     NaN    NaN   
150342      NaN      NaN      NaN     NaN   NaN    NaN    NaN     NaN    NaN   
150343      NaN      NaN      NaN     NaN   NaN    NaN    NaN     NaN    NaN   
150344      NaN      NaN      NaN     NaN   NaN    NaN    NaN     NaN    NaN   
150345      NaN      NaN      NaN     NaN   NaN    NaN    NaN     NaN    NaN   

                   business_id  
0     

In [13]:
GoodForMeal_df = pd.DataFrame(attributes_df['GoodForMeal'])
 
# Apply the custom function to handle the text data
GoodForMeal_df['GoodForMeal'] = GoodForMeal_df['GoodForMeal'].apply(safe_eval)
GoodForMeal_df = pd.concat([GoodForMeal_df, GoodForMeal_df['GoodForMeal'].apply(pd.Series)], axis=1)
GoodForMeal_df = GoodForMeal_df.drop(columns='GoodForMeal')
GoodForMeal_df['business_id'] = attributes_df['business_id']
print(GoodForMeal_df)

       dessert latenight lunch dinner brunch breakfast             business_id
0          NaN       NaN   NaN    NaN    NaN       NaN  Pns2l4eNsfO8kk83dixA6A
1          NaN       NaN   NaN    NaN    NaN       NaN  mpf3x-BjTdTEA3yCZrAYPw
2          NaN       NaN   NaN    NaN    NaN       NaN  tUFrWirKiKi_TAnsVWINQQ
3          NaN       NaN   NaN    NaN    NaN       NaN  MTSW4McQd7CbVtyjqoe9mw
4          NaN       NaN   NaN    NaN    NaN       NaN  mWMc6_wTdE0EUBKIGXDVfA
...        ...       ...   ...    ...    ...       ...                     ...
150341     NaN       NaN   NaN    NaN    NaN       NaN  IUQopTMmYQG-qRtBk-8QnA
150342     NaN       NaN   NaN    NaN    NaN       NaN  c8GjPIOTGVmIemT7j5_SyQ
150343     NaN       NaN   NaN    NaN    NaN       NaN  _QAMST-NrQobXduilWEqSw
150344     NaN       NaN   NaN    NaN    NaN       NaN  mtGm22y5c2UHNXDFAjaPNw
150345     NaN       NaN   NaN    NaN    NaN       NaN  jV_XOycEzSlTx-65W906pg

[150346 rows x 7 columns]


In [14]:
Music_df = pd.DataFrame(attributes_df['Music'])
 
# Apply the custom function to handle the text data
Music_df['Music'] = Music_df['Music'].apply(safe_eval)
Music_df = pd.concat([Music_df, Music_df['Music'].apply(pd.Series)], axis=1)
Music_df = Music_df.drop(columns='Music')
Music_df['business_id'] = attributes_df['business_id']
print(Music_df)

         dj background_music no_music jukebox live video karaoke  \
0       NaN              NaN      NaN     NaN  NaN   NaN     NaN   
1       NaN              NaN      NaN     NaN  NaN   NaN     NaN   
2       NaN              NaN      NaN     NaN  NaN   NaN     NaN   
3       NaN              NaN      NaN     NaN  NaN   NaN     NaN   
4       NaN              NaN      NaN     NaN  NaN   NaN     NaN   
...     ...              ...      ...     ...  ...   ...     ...   
150341  NaN              NaN      NaN     NaN  NaN   NaN     NaN   
150342  NaN              NaN      NaN     NaN  NaN   NaN     NaN   
150343  NaN              NaN      NaN     NaN  NaN   NaN     NaN   
150344  NaN              NaN      NaN     NaN  NaN   NaN     NaN   
150345  NaN              NaN      NaN     NaN  NaN   NaN     NaN   

                   business_id  
0       Pns2l4eNsfO8kk83dixA6A  
1       mpf3x-BjTdTEA3yCZrAYPw  
2       tUFrWirKiKi_TAnsVWINQQ  
3       MTSW4McQd7CbVtyjqoe9mw  
4       mWMc6_wTdE

In [15]:
BestNights_df = pd.DataFrame(attributes_df['BestNights'])
 
# Apply the custom function to handle the text data
BestNights_df['BestNights'] = BestNights_df['BestNights'].apply(safe_eval)
BestNights_df = pd.concat([BestNights_df, BestNights_df['BestNights'].apply(pd.Series)], axis=1)
BestNights_df = BestNights_df.drop(columns='BestNights')
BestNights_df['business_id'] = attributes_df['business_id']
print(BestNights_df)

       monday tuesday friday wednesday thursday sunday saturday  \
0         NaN     NaN    NaN       NaN      NaN    NaN      NaN   
1         NaN     NaN    NaN       NaN      NaN    NaN      NaN   
2         NaN     NaN    NaN       NaN      NaN    NaN      NaN   
3         NaN     NaN    NaN       NaN      NaN    NaN      NaN   
4         NaN     NaN    NaN       NaN      NaN    NaN      NaN   
...       ...     ...    ...       ...      ...    ...      ...   
150341    NaN     NaN    NaN       NaN      NaN    NaN      NaN   
150342    NaN     NaN    NaN       NaN      NaN    NaN      NaN   
150343    NaN     NaN    NaN       NaN      NaN    NaN      NaN   
150344    NaN     NaN    NaN       NaN      NaN    NaN      NaN   
150345    NaN     NaN    NaN       NaN      NaN    NaN      NaN   

                   business_id  
0       Pns2l4eNsfO8kk83dixA6A  
1       mpf3x-BjTdTEA3yCZrAYPw  
2       tUFrWirKiKi_TAnsVWINQQ  
3       MTSW4McQd7CbVtyjqoe9mw  
4       mWMc6_wTdE0EUBKIGXDVfA

In [16]:
HairSpecializesIn_df = pd.DataFrame(attributes_df['HairSpecializesIn'])
 
# Apply the custom function to handle the text data
HairSpecializesIn_df['HairSpecializesIn'] = HairSpecializesIn_df['HairSpecializesIn'].apply(safe_eval)
HairSpecializesIn_df = pd.concat([HairSpecializesIn_df, HairSpecializesIn_df['HairSpecializesIn'].apply(pd.Series)], axis=1)
HairSpecializesIn_df = HairSpecializesIn_df.drop(columns='HairSpecializesIn')
HairSpecializesIn_df['business_id'] = attributes_df['business_id']
print(HairSpecializesIn_df)

       straightperms coloring extensions africanamerican curly kids perms  \
0                NaN      NaN        NaN             NaN   NaN  NaN   NaN   
1                NaN      NaN        NaN             NaN   NaN  NaN   NaN   
2                NaN      NaN        NaN             NaN   NaN  NaN   NaN   
3                NaN      NaN        NaN             NaN   NaN  NaN   NaN   
4                NaN      NaN        NaN             NaN   NaN  NaN   NaN   
...              ...      ...        ...             ...   ...  ...   ...   
150341           NaN      NaN        NaN             NaN   NaN  NaN   NaN   
150342           NaN      NaN        NaN             NaN   NaN  NaN   NaN   
150343           NaN      NaN        NaN             NaN   NaN  NaN   NaN   
150344           NaN      NaN        NaN             NaN   NaN  NaN   NaN   
150345           NaN      NaN        NaN             NaN   NaN  NaN   NaN   

       asian             business_id  
0        NaN  Pns2l4eNsfO8kk83dixA6A

In [17]:
DietaryRestrictions_df = pd.DataFrame(attributes_df['DietaryRestrictions'])
 
# Apply the custom function to handle the text data
DietaryRestrictions_df['DietaryRestrictions'] = DietaryRestrictions_df['DietaryRestrictions'].apply(safe_eval)
DietaryRestrictions_df = pd.concat([DietaryRestrictions_df, DietaryRestrictions_df['DietaryRestrictions'].apply(pd.Series)], axis=1)
DietaryRestrictions_df = DietaryRestrictions_df.drop(columns='DietaryRestrictions')
DietaryRestrictions_df['business_id'] = attributes_df['business_id']
print(DietaryRestrictions_df)

       dairy-free gluten-free vegan kosher halal soy-free vegetarian  \
0             NaN         NaN   NaN    NaN   NaN      NaN        NaN   
1             NaN         NaN   NaN    NaN   NaN      NaN        NaN   
2             NaN         NaN   NaN    NaN   NaN      NaN        NaN   
3             NaN         NaN   NaN    NaN   NaN      NaN        NaN   
4             NaN         NaN   NaN    NaN   NaN      NaN        NaN   
...           ...         ...   ...    ...   ...      ...        ...   
150341        NaN         NaN   NaN    NaN   NaN      NaN        NaN   
150342        NaN         NaN   NaN    NaN   NaN      NaN        NaN   
150343        NaN         NaN   NaN    NaN   NaN      NaN        NaN   
150344        NaN         NaN   NaN    NaN   NaN      NaN        NaN   
150345        NaN         NaN   NaN    NaN   NaN      NaN        NaN   

                   business_id  
0       Pns2l4eNsfO8kk83dixA6A  
1       mpf3x-BjTdTEA3yCZrAYPw  
2       tUFrWirKiKi_TAnsVWINQQ  
3  

In [19]:
Parking_df.to_sql(name = "businessparking", con = engine, if_exists='replace', index=True, chunksize=50000)
Ambience_df.to_sql(name = "ambience", con = engine, if_exists='replace', index=True, chunksize=50000)
GoodForMeal_df.to_sql(name = "goodformeal", con = engine, if_exists='replace', index=True, chunksize=50000)
Music_df.to_sql(name = "music", con = engine, if_exists='replace', index=True, chunksize=50000)
BestNights_df.to_sql(name = "bestnights", con = engine, if_exists='replace', index=True, chunksize=50000)
HairSpecializesIn_df.to_sql(name = "hairspecializesin", con = engine, if_exists='replace', index=True, chunksize=50000)
DietaryRestrictions_df.to_sql(name = "dietaryrestrictions", con = engine, if_exists='replace', index=True, chunksize=50000)

150346